# Update location fields


In [3]:
import pymongo
import pandas as pd
import numpy as np
import random

In [4]:
def db_connection(collection_name): 
    # connect to mLab DB
    try:
        with open("../credentials/mlab_credentials.txt", 'r', encoding='utf-8') as f:
            [name,password,url,dbname]=f.read().splitlines()
            db_conn = pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            print ("DB connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to DB: %s" % e) 
        
    db = db_conn[dbname]
    collection = db[collection_name]
    
    return collection

In [16]:
db_tweets = db_connection("tweets")
print(db_tweets.count())

DB connected successfully!!!


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


6153


In [13]:
db_tweets_bckp = db_connection("tweets_bckp_181205")
print(db_tweets_bckp.count())

DB connected successfully!!!


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


2618


In [18]:
# uncomment to delete
#result = db_tweets_bckp.delete_many({})
#print(result.deleted_count, " documents deleted")

2618  documents deleted


In [19]:
print(db_tweets_bckp.count())

0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [20]:
tweets_list = [tweet for tweet in db_tweets.find()]

In [21]:
len(tweets_list)

6153

In [22]:
db_tweets_bckp.insert_many(tweets_list)
print(db_tweets_bckp.count())

6153


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


In [24]:
for tweet in tweets_list:
    if tweet["coordinates"] is not None and tweet["coordinates"]["type"] == "Point":
        tweet['lat'] = tweet["coordinates"]["coordinates"][1]
        tweet['lon'] = tweet["coordinates"]["coordinates"][0]
    else:
        tweet['lat'] = None
        tweet['lon'] = None
    db_tweets.replace_one({"id":tweet["id"]}, tweet, upsert = True)

In [25]:
location_query = {
    "lat": {
        "$ne": None
    }
}

In [27]:
for tweet in db_tweets.find(location_query)[:10]:
    print(tweet["coordinates"],tweet['lat'],tweet['lon'])

{'type': 'Point', 'coordinates': [-87.71474, 41.96196]} 41.96196 -87.71474
{'type': 'Point', 'coordinates': [-73.999952, 40.754375]} 40.754375 -73.999952
{'type': 'Point', 'coordinates': [-73.9819456, 40.7594726]} 40.7594726 -73.9819456
{'type': 'Point', 'coordinates': [-73.99453567, 40.7413536]} 40.7413536 -73.99453567
{'type': 'Point', 'coordinates': [-74.0064, 40.7142]} 40.7142 -74.0064
{'type': 'Point', 'coordinates': [-74.0059413, 40.7127837]} 40.7127837 -74.0059413
{'type': 'Point', 'coordinates': [-77.0367, 38.8951]} 38.8951 -77.0367
{'type': 'Point', 'coordinates': [-76.9716034, 38.91988017]} 38.91988017 -76.9716034
{'type': 'Point', 'coordinates': [-76.9716034, 38.91988017]} 38.91988017 -76.9716034
{'type': 'Point', 'coordinates': [-76.9912, 38.933]} 38.933 -76.9912
